# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print("Databases:")
pprint(mongo.list_database_names())

Databases:
['admin',
 'config',
 'fruit_db',
 'local',
 'met',
 'myDatabase',
 'petsitly_marketing',
 'test',
 'test_db',
 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print("\nCollections:")
pprint(db.list_collection_names())


Collections:
['establishments']


In [6]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [7]:
# Create a dictionary for the new restaurant data

new_restaurant = {
    "BusinessName": "The Food Place",
    "BusinessType": "Restaurant",
    "BusinessTypeID": 12345,
    "AddressLine1": "123 Food Street",
    "AddressLine2": "",
    "AddressLine3": "",
    "PostCode": "AB12 3CD",
    "PhoneNumber": "01234 567890",
    "RatingValue": 5,
    "RatingKey": 1,
    "RatingDate": "2023-10-01",
    "LocalAuthorityBusinessID": 1234567890,
}

In [8]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)



InsertOneResult(ObjectId('67eb49f578aaae6ef687abaa'), acknowledged=True)

In [9]:
# Check that the new restaurant was inserted
print("\nInserted New Restaurant:")
pprint(establishments.find_one({"BusinessName": "The Food Place"}))




Inserted New Restaurant:
{'AddressLine1': '123 Food Street',
 'AddressLine2': '',
 'AddressLine3': '',
 'BusinessName': 'The Food Place',
 'BusinessType': 'Restaurant',
 'BusinessTypeID': 1,
 'LocalAuthorityBusinessID': 1234567890,
 'PhoneNumber': '01234 567890',
 'PostCode': 'AB12 3CD',
 'RatingDate': '2023-10-01',
 'RatingKey': 1,
 'RatingValue': 5,
 '_id': ObjectId('67eb493974547eac6b983991')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [10]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1}
)
print("\nBusinessTypeID for 'Restaurant/Cafe/Canteen':")
pprint(business_type)

# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, '_id': 0}
)

print("\nBusinessTypeID for 'Restaurant/Cafe/Canteen':")

pprint(business_type)



BusinessTypeID for 'Restaurant/Cafe/Canteen':
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('67e63be0194e3f972c725097')}

BusinessTypeID for 'Restaurant/Cafe/Canteen':
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [11]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"BusinessName": "The Food Place"},
    {"$set": {"BusinessTypeID": business_type["BusinessTypeID"]}}
)



UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [12]:
# Confirm that the new restaurant was updated

print("\nUpdated New Restaurant:")
pprint(establishments.find_one({"BusinessName": "The Food Place"}))



Updated New Restaurant:
{'AddressLine1': '123 Food Street',
 'AddressLine2': '',
 'AddressLine3': '',
 'BusinessName': 'The Food Place',
 'BusinessType': 'Restaurant',
 'BusinessTypeID': 1,
 'LocalAuthorityBusinessID': 1234567890,
 'PhoneNumber': '01234 567890',
 'PostCode': 'AB12 3CD',
 'RatingDate': '2023-10-01',
 'RatingKey': 1,
 'RatingValue': 5,
 '_id': ObjectId('67eb493974547eac6b983991')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [13]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("\nNumber of documents with LocalAuthorityName 'Dover':")
pprint(dover_count)


Number of documents with LocalAuthorityName 'Dover':
0


In [14]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [15]:
# Check if any remaining documents include Dover
remaining_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("\nRemaining documents with LocalAuthorityName 'Dover':")

pprint(remaining_dover)



Remaining documents with LocalAuthorityName 'Dover':
0


In [16]:
# Check that other documents remain with 'find_one'
print("\nRemaining documents:")
pprint(establishments.find_one({"BusinessName": "The Food Place"}))



Remaining documents:
{'AddressLine1': '123 Food Street',
 'AddressLine2': '',
 'AddressLine3': '',
 'BusinessName': 'The Food Place',
 'BusinessType': 'Restaurant',
 'BusinessTypeID': 1,
 'LocalAuthorityBusinessID': 1234567890,
 'PhoneNumber': '01234 567890',
 'PostCode': 'AB12 3CD',
 'RatingDate': '2023-10-01',
 'RatingKey': 1,
 'RatingValue': 5,
 '_id': ObjectId('67eb493974547eac6b983991')}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [17]:
# covert latitude and longitude to floats
establishments.update_many(
    {"geocode.latitude": {"$type": "string"}},
    [{"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}]
)

establishments.update_many(
    {"geocode.longitude": {"$type": "string"}},
    [{"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}]
)

# Check that the latitude and longitude were converted to floats
print("\nLatitude and Longitude converted to floats:")
pprint(establishments.find_one({"BusinessName": "The Food Place"}, {"geocode.latitude": 1, "geocode.longitude": 1}))




Latitude and Longitude converted to floats:
{'_id': ObjectId('67eb493974547eac6b983991')}


Use `update_many` to convert `RatingValue` to integer numbers.

In [18]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [ ]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

In [21]:
# Check that the coordinates and rating value are now numbers
print("\nUpdated coordinates and rating value:")
pprint(establishments.find_one({"BusinessName": "The Food Place"}, {"geocode.latitude": 1, "geocode.longitude": 1, "RatingValue": 1}))
# Check that the rating value is now an integer
print("\nUpdated rating value:")
pprint(establishments.find_one({"BusinessName": "The Food Place"}, {"RatingValue": 1}))



Updated coordinates and rating value:
{'RatingValue': 5, '_id': ObjectId('67eb493974547eac6b983991')}

Updated rating value:
{'RatingValue': 5, '_id': ObjectId('67eb493974547eac6b983991')}
